# ARIMA Single Model

---

In [20]:
# DataFrame
import pandas as pd
import numpy as np
import random
from datetime import datetime, date

# Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Visualization
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')

# Save the log
import os
import pickle
import time
from datetime import timedelta

# LGBM
import lightgbm as lgbm

# Metric 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

In [21]:
# Minus
matplotlib.rcParams['axes.unicode_minus'] = False
# 한글
plt.rc('font', family='NanumGothic')

## Data Explore

In [22]:
df = pd.read_csv("../Data/JW_capstone.csv")
df['Date'] = pd.to_datetime(df['Date'])

---

### Split the train and test set
- Input
     data: dataframe with dates and Demand data
     
- output
    - train: ~ 2012-06-30
    
    - test :  2022-07-01 ~ 2012-12-31

In [23]:
# train과 test로 데이터 split
def split_data(product_df):
    
    train_df = product_df[product_df['Date']<'2022-07-01'].reset_index(drop=True)
    test_df = product_df[product_df['Date']>='2022-07-01'].reset_index(drop=True)
    return train_df, test_df

## ARIMA
- Rolling forecast 방식으로 input값으로 테스트 기간의 값이 들어가서, 1일 후를 예측한 것을 총 6개월 이어붙임

In [24]:
product_df = df[df['Product']=="리바로정"].reset_index(drop=True)

In [25]:
train_df, test_df = split_data(product_df)
predictions = []
lgbm_model = lgbm.LGBMRegressor()
lgbm_model.fit(
        x_train,
        y_train
)
preds = lgbm_gbr.predict(x_valid)

for test_date in test_df['Date']:
    print(test_date)
    # test_date -= timedelta(days=1)
    # Use data up to the day before the test date to train the model
    train_until_test_df = product_df[product_df['Date'] < test_date]
    
    # fitting the model with best(p,d,q)
    best_model_fit = lgbm_model.fit(train_until_test_df['y'])

    # 1일 뒤의 값을 예측하고, 예측값에 새롭게 추가
    prediction = best_model_fit.predict(n_periods=1).reset_index(drop=True)[0]
    predictions.append(max(0, prediction))  # replace negative predictions with 0
    
# Create a DataFrame to hold the result
res_df = test_df.copy()
res_df['Pred'] = predictions
res_df.set_index('Date', inplace=True)

NameError: name 'x_train' is not defined

In [15]:
# Rolling Forecasting
def ARIMA_single(product_df):

    train_df, test_df = split_data(product_df)

    predictions = []
    best_model = auto_arima(train_df['y'], 
                            start_p=1, start_q=1,
                            max_p=5, max_q=5, 
                            max_d=2, trace=True,
                            suppress_warnings=True)
    
    for test_date in test_df['Date']:
        print(test_date)
        # test_date -= timedelta(days=1)
        # Use data up to the day before the test date to train the model
        train_until_test_df = product_df[product_df['Date'] < test_date]
        
        # fitting the model with best(p,d,q)
        best_model_fit = best_model.fit(train_until_test_df['y'])
    
        # 1일 뒤의 값을 예측하고, 예측값에 새롭게 추가
        prediction = best_model_fit.predict(n_periods=1).reset_index(drop=True)[0]
        predictions.append(max(0, prediction))  # replace negative predictions with 0

    # Create a DataFrame to hold the result
    res_df = test_df.copy()
    res_df['Pred'] = predictions
    res_df.set_index('Date', inplace=True)

    # res_df: ['y'','Pred'] index='Date'
    return best_model, res_df

In [9]:
product_df = df[df['Product']=="리바로정"].reset_index(drop=True)

In [10]:
train_df, test_df = split_data(product_df)

predictions = []

# We assume that product_df has a 'Date' column and a 'y' column
# If you have more features, you should add them here.
X_train = train_df['Date'].astype(int).values.reshape(-1, 1)
y_train = train_df['y'].values

# Create LGBM Dataset
lgb_train = lgb.Dataset(X_train, y_train)

# Define hyperparameters for LightGBM
# Note: You might want to tune these hyperparameters or use a hyperparameter tuning algorithm
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

# Train LightGBM model
# You might want to adjust num_boost_round and early_stopping_rounds
model = lgb.train(params, lgb_train, num_boost_round=1000, valid_sets=[lgb_train], early_stopping_rounds=10, verbose_eval=False)

for test_date in test_df['Date']:
    print(test_date)
    # Convert date to int and reshape for prediction
    X_test = np.array([[test_date.astype(int)]])
    prediction = model.predict(X_test)[0]
    predictions.append(max(0, prediction))  # replace negative predictions with 0

# Create a DataFrame to hold the result
res_df = test_df.copy()
res_df['Pred'] = predictions
res_df.set_index('Date', inplace=True)

TypeError: Converting from datetime64[ns] to int32 is not supported. Do obj.astype('int64').astype(dtype) instead

## Save & Load the model

In [49]:
def save_model(product_code, best_model):
    today = date.today()
    folder_path = 'Result/Single_ARIMA_Result/Model'
    file_name = f'{product_code}_{today.month:02d}{today.day:02d}.pkl'
    save_path = os.path.join(folder_path, file_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    # 객체를 pickle 파일로 저장
    with open(save_path, 'wb') as f:
        pickle.dump(best_model, f)
    return best_model

In [50]:
# 학습된 모델 pickle파일에서 불러오기
def load_model(file_name):
    file_path = f'Result/Single_ARIMA_Result/Model/{file_name}'
    
    with open(file_path, 'rb') as file:
        model_dict= pickle.load(file)
    
    return best_model

## Plot the result

In [57]:
def actual_pred_plot(product_code, res_df, metric_df):
    today = date.today()
    """
    Plot the actual vs predition and save the figure in the given directory
    """
    save_path = os.path.join("Result", "Single_ARIMA_Result", product_code)
    save_name = f'{product_code}_all_result'
    
    title = f"Pred Actual Plot - {product_code}"
    actual = res_df['y']
    pred = res_df['Pred']

    # Plot   
    plt.figure(figsize=(16, 8))
    plt.title(title, fontsize=20)
    plt.xlabel("Date", fontsize=14)
    plt.ylabel("Order Demand", fontsize=14)
    plt.plot(actual, label ='Actual', color='r', marker='o', ms=3)
    plt.plot(pred, label='Prediction', color='b', marker='o', ms=3)
    plt.legend(loc="upper right")
        
    # Plot 결과 저장
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # save the figure
    today_date = f'_{today.month:02d}{today.day:02d}'
    plt.savefig(os.path.join(save_path, save_name+'.png'))
    plt.show()
    
    # 전체 결과에 대한 Metric도 함께 저장
    metric_df.to_csv(os.path.join(save_path, save_name+'.csv'), encoding="utf-8-sig")
    plt.close('all') # close all figures to free up memory

## Metrics

In [52]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual+1)))

# 정규화 된 지표
def nrmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, squared=False)
    target_mean = np.mean(y_true)
    nrmse = mse / target_mean
    return nrmse

# 정규화 된 지표
def nmae(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    target_mean = np.mean(y_true)
    nmae = mae / target_mean
    return nmae

In [53]:
def calculate_metrics(product_code, res_df):

    actual = res_df['y']
    pred = res_df['Pred']

    # 메트릭 계산
    MAPE = mape(actual, pred) 
    RMSE = mean_squared_error(actual, pred)**0.5 
    MAE = mean_absolute_error(actual,pred) 
    NRMSE = nrmse(actual,pred) 
    NMAE = nmae(actual,pred) 
    R2 = r2_score(actual, pred)

    # 계산된 메트릭을 데이터프레임에 추가
    metric_df = pd.DataFrame({'MAPE':[round(MAPE, 4)],
                           'RMSE':[round(RMSE, 4)],
                           'MAE':[round(MAE, 4)],
                           'NRMSE':[round(NRMSE, 4)],
                           'NMAE':[round(NMAE, 4)],
                           'R2': [round(R2, 4)]},
                            index= [product_code])

    return metric_df

---

## Check the Result

In [54]:
def execute_single_ARIMA(product_code):
    start_time = time.time()
    
    product_code = product_code # 예측하고자 하는 코드 입력
    product_df = df[df['Product']== product_code].reset_index(drop=True)

    # Prophet 단일 모델
    best_model, res_df = ARIMA_single(product_df) 
    save_model(product_code, best_model)
    # 모델 Metric과 Pred_Actual Plot 저장
    metric_df= calculate_metrics(product_code, res_df)
    actual_pred_plot(product_code, res_df, metric_df)
    
    # 실행시간 확인
    elapsed_time_seconds = time.time() - start_time
    elapsed_time_minutes = elapsed_time_seconds / 60
    print("실행 시간: {:.2f} 분".format(elapsed_time_minutes))
    return metric_df

---

## Whole Process
    - product_code에 str으로 예측하고자 하는 코드를 입력

In [99]:
for code in df['Product'].unique():
    print("==================================")
    print(f"========== { code } ==========")
    print("==================================")
    execute_single_ARIMA(code)

========== Product_0025 ==========
실행 시간: 0.03 분
                  MAPE      RMSE       MAE   NRMSE
Product_0025  291.0962  965.0509  641.5268  1.9693
========== Product_0739 ==========
실행 시간: 0.11 분
                MAPE     RMSE      MAE   NRMSE
Product_0739  0.2932  46.9636  19.2149  2.4441
========== Product_0901 ==========
실행 시간: 0.49 분
                 MAPE     RMSE      MAE   NRMSE
Product_0901  12.5931  89.6329  45.8805  2.2913
========== Product_1154 ==========
실행 시간: 0.02 분
                  MAPE       RMSE        MAE   NRMSE
Product_1154  944.7914  2806.3135  1845.0757  2.2603
========== Product_1248 ==========
실행 시간: 0.02 분
                    MAPE         RMSE          MAE   NRMSE
Product_1248  55738.2977  233395.9957  167441.9966  1.5898
========== Product_1295 ==========
실행 시간: 0.58 분
                   MAPE        RMSE        MAE   NRMSE
Product_1295  4108.0797  68438.9556  44018.159  0.9371
========== Product_1378 ==========
실행 시간: 0.11 분
                    MAPE       

In [ ]:
#load_model('Product_0739_0503.pkl')